---
**Author:** Arifa Kokab  
**For:** AAI-540 Machine Learning Operations  
**Institution:** University of San Diego

# Automated Model Registration and Governance Pipeline for Facial Expression Analysis (FER+ & RAF-DB) – MobileNetV2

## Introduction

This notebook demonstrates a fully automated approach to model registration, documentation, and governance for a facial expression analysis system based on the MobileNetV2 architecture. The model is trained on the FER+ and RAF-DB datasets to classify emotions from facial images for neuromarketing applications.

Following best practices in MLOps, this workflow uses AWS SageMaker’s Model Registry and Model Card features, automating:

- **Creation of a Model Package Group** to manage model versions
- **Registration of a new Model Package** with reproducible metadata
- **Generation and approval of a Model Card** for transparent documentation

By scripting these steps, we satisfy core **CI/CD (Continuous Integration/Continuous Deployment) requirements** for ML systems: enabling reproducible, auditable, and fully automated model lifecycle management. This ensures that each new version of the model is registered and documented in a consistent, scalable, and production-ready manner.

## 1. Environment and SageMaker Client Setup

This section initializes the required libraries and sets up the SageMaker Boto3 client for automating model registration and documentation.

In [1]:
import boto3
import json

client = boto3.client("sagemaker")

## 2. Create and Describe Model Package Group

We first create a Model Package Group in SageMaker Model Registry. This group organizes all versions of our facial expression analysis models for tracking and lifecycle management.

In [2]:
model_group_name = "FacialExpressionAnalysis-ModelGroup"

response = client.create_model_package_group(
    ModelPackageGroupName=model_group_name,
    ModelPackageGroupDescription="Model Group for Facial Expression Analysis - MobileNetV2"
)
print("✓ Model Group created:", model_group_name)

✓ Model Group created: FacialExpressionAnalysis-ModelGroup


In [3]:
response = client.describe_model_package_group(ModelPackageGroupName=model_group_name)
print(response)

{'ModelPackageGroupName': 'FacialExpressionAnalysis-ModelGroup', 'ModelPackageGroupArn': 'arn:aws:sagemaker:us-east-1:301806113644:model-package-group/FacialExpressionAnalysis-ModelGroup', 'ModelPackageGroupDescription': 'Model Group for Facial Expression Analysis - MobileNetV2', 'CreationTime': datetime.datetime(2025, 6, 26, 2, 34, 38, 974000, tzinfo=tzlocal()), 'CreatedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:301806113644:user-profile/d-5xmcc0ygmqtr/arifakokab', 'UserProfileName': 'arifakokab', 'DomainId': 'd-5xmcc0ygmqtr', 'IamIdentity': {'Arn': 'arn:aws:sts::301806113644:assumed-role/LabRole/SageMaker', 'PrincipalId': 'AROAUMRIL2NWJIEA5CLZN:SageMaker'}}, 'ModelPackageGroupStatus': 'Completed', 'ResponseMetadata': {'RequestId': '621dbd07-0d9e-4d94-98c5-260b2c004b67', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '621dbd07-0d9e-4d94-98c5-260b2c004b67', 'content-type': 'application/x-amz-json-1.1', 'content-length': '646', 'date': 'Thu, 26 Jun 2025 02:34:42 GM

## 3. Register a Model Package to the Model Group

This section registers a new version of the trained MobileNetV2 model (with S3 path and container image) to the Model Package Group for deployment and production tracking.

In [6]:
# Define the container image URI and model artifact S3 URL
container_image_uri = "763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-inference:1.9.1-cpu-py38"
model_data_url = "s3://sagemaker-us-east-1-301806113644/FacialExpressionAnalysisFinal/model.tar.gz"

# Model package registration input
model_package_input = {
    "ModelPackageGroupName": model_group_name,
    "ModelPackageDescription": "Production-ready model for FER+ and RAF-DB emotion classification",
    "InferenceSpecification": {
        "Containers": [
            {
                "Image": container_image_uri,
                "ModelDataUrl": model_data_url
            }
        ],
        "SupportedContentTypes": ["application/x-image"],
        "SupportedResponseMIMETypes": ["application/json"]
    },
    "ModelApprovalStatus": "Approved"
}

response = client.create_model_package(**model_package_input)
print("✓ Model Package registered.")

✓ Model Package registered.


In [7]:
# Save the Model Package ARN from the registration response
model_package_arn = response['ModelPackageArn']

# Describe the model package using its ARN
describe_response = client.describe_model_package(ModelPackageName=model_package_arn)

# Print the model package details
print("✓ Model Package details:")
print(describe_response)

✓ Model Package details:
{'ModelPackageGroupName': 'FacialExpressionAnalysis-ModelGroup', 'ModelPackageVersion': 1, 'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:301806113644:model-package/FacialExpressionAnalysis-ModelGroup/1', 'ModelPackageDescription': 'Production-ready model for FER+ and RAF-DB emotion classification', 'CreationTime': datetime.datetime(2025, 6, 26, 2, 40, 23, 530000, tzinfo=tzlocal()), 'InferenceSpecification': {'Containers': [{'Image': '763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-inference:1.9.1-cpu-py38', 'ImageDigest': 'sha256:327d9112f93764b3d94edf7e2629d9a4d4dc410ccce0e7d8e3a8f4b4d9cdc91c', 'ModelDataUrl': 's3://sagemaker-us-east-1-301806113644/FacialExpressionAnalysisFinal/model.tar.gz', 'ModelDataETag': 'a3d508a38fdcb8d2ecc5992990707e09-2'}], 'SupportedContentTypes': ['application/x-image'], 'SupportedResponseMIMETypes': ['application/json']}, 'ModelPackageStatus': 'Completed', 'ModelPackageStatusDetails': {'ValidationStatuses': [], 'ImageScanStat

## 4. Create and Approve a Model Card

This section creates a Model Card documenting the model's overview, training details, evaluation results, and links it to the registered model package for governance and auditability.

In [9]:
# Define model card content
model_card_content = {
    "model_overview": {
        "model_description": "MobileNetV2 model trained to classify facial expressions into 7 emotions using FERPlus and RAF-DB datasets.",
        "model_owner": "Arifa Kokab",
        "problem_type": "Image Classification",
        "algorithm_type": "Convolutional Neural Network"
    },
    "intended_uses": {
        "intended_uses": "Neuromarketing teams will use this model to analyze emotional responses from facial expressions across video content."
    },
    "training_details": {
        "training_observations": "The model was trained using stratified and balanced data splits from FERPlus and RAF-DB. Image augmentation and class weights were used."
    },
    "evaluation_details": [
        {
            "name": "Validation Accuracy",
            "evaluation_job_arn": "arn:aws:sagemaker:us-east-1:301806113644:transform-job/facial-expression-eval-job",
            "evaluation_observation": "Validation accuracy achieved 89.6% on combined FERPlus and RAF-DB validation set."
        },
        {
            "name": "Macro F1 Score",
            "evaluation_job_arn": "arn:aws:sagemaker:us-east-1:301806113644:transform-job/facial-expression-eval-job",
            "evaluation_observation": "Macro average F1 score was 0.831 as calculated using sklearn metrics."
        }
    ],
    "model_package_details": {
        "model_package_group_name": "FacialExpressionAnalysisModelGroup",
        "model_package_version": 1
    }
}

# Create the model card
response = client.create_model_card(
    ModelCardName="FacialExpressionAnalysis-Card",
    Content=json.dumps(model_card_content),
    ModelCardStatus="Approved"
)

print("✓ Model card created and linked to model package:")
print("ModelCardName:", response["ModelCardArn"])

✓ Model card created and linked to model package:
ModelCardName: arn:aws:sagemaker:us-east-1:301806113644:model-card/FacialExpressionAnalysis-Card


---

## Results: Model Registration and Model Card Creation

- **Model Package Registered:**  
  Model registered in SageMaker Model Registry under group `FacialExpressionAnalysis-ModelGroup` (Version 1).  
  **ModelPackageArn:**  
  `arn:aws:sagemaker:us-east-1:301806113644:model-package/FacialExpressionAnalysis-ModelGroup/1`

- **Model Card Created:**  
  Comprehensive model card generated and linked to the registered model package for documentation, compliance, and governance.  
  **ModelCardArn:**  
  `arn:aws:sagemaker:us-east-1:301806113644:model-card/FacialExpressionAnalysis-Card`

---

**CI/CD Automation Note:**  
These steps were performed entirely by script, enabling this workflow to be integrated into a CI/CD pipeline (such as a scheduled job, GitHub Actions, or AWS CodePipeline).  
By registering models and model cards programmatically, we ensure every new model version is:
- Registered, versioned, and auditable
- Documented with transparent evaluation and intended use
- Governed according to MLOps best practices

This approach meets industry standards for reproducible and automated machine learning operations.